<a href="https://colab.research.google.com/github/DallasAutumn/datawhale_salons/blob/master/NLP_s8_task4(a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-s8-task4 文本分类

## 1. 算法原理

### 1.1 朴素贝叶斯的原理

朴素贝叶斯分类器基于“属性条件独立性假设”：对已知类别，假设所有属性相互独立。利用贝叶斯公式计算后验概率：
$$P(c|x) = \frac{P(c)P(x|c)}{P(x)} = \frac{P(c)}{P(x)} \prod_{i=1}^d P(x_i|c)$$
其中d为属性数目，$x_i$为$\textbf{x}$在第i个属性上的取值。

由于对所有类别来说$P(\textbf{x})$相同，那么我们要训练的朴素贝叶斯分类器表达式就是：
$$h_{nb}(\textbf{x}) = \underset{c \in \mathcal{Y}}{\operatorname{argmax}} P(c) \prod_{i=1}^d P(x_i|c)$$

### 1.2 支持向量机原理

给定一组样本$D = \lbrace{(x_1,y_1), (x_2, y_2),\ldots, (x_m,y_m)}\rbrace, y_i \in \lbrace{-1, +1}\rbrace$，要找到样本空间中的一个划分超平面$\omega^Tx+b=0$，将不同类别的样本分开。

假设超平面能够将样本正确分类，即：
$$
\begin{cases}
{\omega^Tx+b \geq +1,}&{y_i=+1}\\
{\omega^Tx+b \leq -1,}&{y_i=-1}
\end{cases}
$$

距离超平面最近的几个样本点使得等号成立，他们被称为“支持向量（support vector），两个异类支持向量到超平面的距离之和为
$$\gamma = \frac{2}{\parallel{\omega}\parallel}$$，它们被称为”间隔（margin）“。

 欲找到具有”最大间隔（maximum marginal）的超平面，也就是要找到能满足约束的参数$\omega和b$，使得$\gamma$最大，即
 $$
 \begin{align}
 &\underset{\omega, b}{\operatorname{max}}\frac{2}{\parallel{\omega}\parallel} \\
 {s.t.\quad}&{y_i(w^Tx_i+b)}{\geq 1,} & i=1,2,\ldots,m.
 \end{align}
 $$

### 1.3 主题模型

#### 1.3.1 pLSA





pLSA是一个用生成模型来建模文章的生成过程。假设有K个主题，M篇文章；对语料库中的任意文章d，假设该文章有N个词，则对于其中的每一个词，我们首先选择一个主题z，然后在当前主题的基础上生成一个词w。

生成主题z和词w的过程遵照一个确定的概率分布。设在文章d中生成主题z的概率为$p(z|d)$，在选定主题的条件下生成词w的概率为$p(w|z)$，则给定文章d，生成词w的概率可以写成：$p(w|d)=\sum_z p(w|z,d)p(z|d)$。在这里我们做一个简化，假设给定主题z的条件下，生成词w的概率是与特定的文章无关的，则公式可以简化为：$p(w|d)=\sum_z p(w|z)p(z|d)$。整个语料库中的文本生成概率可以用似然函数表示为
$$L=\prod_m^M\prod_n^N p(d_m,w_n)^{c(d_m,w_n)}$$
其中$p(d_m,w_n)$是在第m篇文章中$d_m$中，出现单词$w_n$的概率，与上下文中$p(w|d)$的含义是相同的，只是换了一种符号表达；$c(d_m,w_n)$是在第m篇文章$d_m$中，单词$w_n$出现的次数。

于是，对数似然可以写成：
$$l=\sum_m^M\sum_n^N c(d_m,w_n)\log{p(d_m,w_n)}=\sum_m^M\sum_n^N c(d_m,w_n)\log{\sum_k^Kp(d_m)p(z_k|d_m)p(w_n|z_k)}$$

在上面的公式中，定义在文章上的主题分布$p(z_k|d_m)$和定义在主题上的词分布$p(w_n|z_k$是待估计的参数。我们要让对数似然最大化。

#### 1.3.2 LDA

LDA可以看作是pLSA的贝叶斯版本，其文本生成过程与pLSA基本相同，不同的是为主题分布和词分布分别加了两个狄利克雷先验。之所以选择狄利克雷分布作为先验分布，是因为它为多项式分布的共轭先验分布，后验概率依然服从狄利克雷分布，这样做可以为计算提供便利。

语料库的生成过程为：对文本库中每一篇文章$d_i$，采用以下操作:

（1）从超参数为$\alpha$的狄利克雷分布中抽样生成文档$d_i$的主题分布$\theta_i$。

（2）对文档$d_i$中的每一个词进行以下3个操作。
 - 从代表主题的多项式分布$\theta_i$中抽样生成它所对应的主题$z_{ij}$
 - 从超参数为$\beta$的狄利克雷分布中抽样生成主题$z_{ij}$对应的词分布$\Psi_{z_{ij}}$
 - 从代表词的多项式分布$\Psi_{z_{ij}}$中抽样生成词$w_{ij}$
 
 
 我们要求解出主题分布$\theta_i$以及词分布$\Psi_{z_{ij}}$的期望，可以用吉布斯采样（Gibbs Sampling）的方式实现。首先随机给定每个单词的主题，然后在其他变量固定的情况下，根据转移概率抽样生成每个单词的新主题。对于每个单词来说，转移概率可以理解为：给定文章中的所有单词以及除自身意外其他所有单词的主题，在此条件下该单词对应为各个新主题的概率。然后，经过反复迭代，我们可以根据收敛后的采样结果计算主题分布和词分布的期望。

## 2。实战（见b笔记本）